In [2]:
import tensorflow as tf
import numpy as np
import os
import utils

In [3]:
def kernel(x1,x2,l):
    return np.exp(-1.0/l**2*np.sum((np.expand_dims(x1,axis=2) - np.expand_dims(x2,axis=1))**2, axis = 3))

def GP(X,A,x,l):
    k_xX = kernel(x,X,l)
    return np.squeeze(np.matmul(k_xX,  A),axis=(2,))

def normalize(minv, maxv, y):
    return 2*(y-minv)/(maxv-minv)-1.0

In [4]:
def gen_data(n_data, dim, n_gp_samples, n_mc_samples, l):
    X = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples, dim))
    Y = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples))

    K_XX = kernel(X,X,l)
    A = np.linalg.solve(K_XX, np.expand_dims(Y,axis=2))
    
    mc_samples = np.random.uniform(low = -1.0, high = 1.0, size = [1, n_mc_samples, dim])
    y = GP(X,A,mc_samples,l)

    min_vals = np.min(y, axis = 1).reshape(n_data,1)
    max_vals = np.max(y, axis = 1).reshape(n_data,1)

    return (X, A, min_vals, max_vals)

In [6]:
data_dir = utils.loadConfig()['DATA_DIR']
def run_gen_data_for(dim, no_training, no_testing, n_gp_samples, n_mc_samples, length ):
    no_data = [no_training,no_testing]
    for i in range(1):
        X, A, minv, maxv = gen_data( no_data[i], dim, n_gp_samples, n_mc_samples, length)
        s = 'training' if i == 0 else 'testing'
        directory = utils.getDataPath(dim, s)
        if not os.path.exists(directory):
            os.makedirs(directory)

        np.save( directory + "/X", X)
        np.save( directory + "/A", A)
        np.save( directory + "/minv", minv)
        np.save( directory + "/maxv", maxv)

In [7]:
run_gen_data_for(2, 2000, 200, 6, 1000, 0.4714045207910317) # generate for 1D